In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import os


In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.getcwd() # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

#visualize the data
print(df.head())
print(df.info())

                                             context  \
0  Ondertussen is de eerstelijnszone BruZEL al me...   
1  Ondertussen is de eerstelijnszone BruZEL al me...   
2  Ondertussen is de eerstelijnszone BruZEL al me...   
3  Ondertussen is de eerstelijnszone BruZEL al me...   
4  Ondertussen is de eerstelijnszone BruZEL al me...   

                                            question  statistical  \
0  Zoals alle eerstelijnszones kreeg ook BruZEL h...            0   
1                            a)Wat liep er moeilijk?            0   
2  Met welke  uitdagingen  werd BruZEL het afgelo...            0   
3      Hoe kunnen  die uitdagingen worden aangepakt?            0   
4                                b)Wat liep er goed?            0   

                        theme    file_name  
0  Brussel en de Vlaamse Rand  1752898.txt  
1  Brussel en de Vlaamse Rand  1752898.txt  
2  Brussel en de Vlaamse Rand  1752898.txt  
3  Brussel en de Vlaamse Rand  1752898.txt  
4  Brussel en de Vlaams

In [3]:
# Drop unnecessary columns
if "file_name" in df.columns:
    df = df.drop(columns=["file_name"])
    df = df.drop(columns=["statistical"])


# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)

# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s+', ' ', text)  # Remove patterns like 'a)', 'b)', etc.
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    return text

#df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = (df["question"]).apply(clean_text)
# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Merge back with the original dataframe to get all rows with these 'clean_text'
filtered_df = df[df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]
# Exclude rows with these 'clean_text' from the original dataframe
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]


# ✅ Now: drop rare themes using original theme names
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]


C:\Users\jefva\AppData\Local\Temp\ipykernel_14888\149515185.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# 1. Encode labels
le = LabelEncoder()
y = le.fit_transform(df["theme"])
X = df["clean_text"]  # Use the combined, cleaned text

# 2. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 3. Create and train pipeline
nb_model = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), max_features=10000),
    MultinomialNB()
)

nb_model.fit(X_train, y_train)

# 4. Predict
y_pred = nb_model.predict(X_test)

# 5. Decode labels back to original strings (optional, for readable report)
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred)

# 6. Evaluate
print(classification_report(y_test_labels, y_pred_labels, zero_division=0))


                                                                     precision    recall  f1-score   support

                                                      Armoedebeleid       0.00      0.00      0.00        86
                                                          Begroting       0.82      0.18      0.30       259
                                         Brussel en de Vlaamse Rand       0.50      0.01      0.01       156
                                                 Buitenlands beleid       0.90      0.45      0.60       304
                                                            Cultuur       0.68      0.19      0.30       533
                                                      Dierenwelzijn       0.88      0.20      0.32       374
                                                           Economie       0.52      0.30      0.38       692
                                                            Energie       0.67      0.26      0.37       661
                  